In [2]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print(" No GPU! Go to Runtime -> Change runtime type -> GPU")

PyTorch version: 2.9.0+cu126
CUDA available: False
 No GPU! Go to Runtime -> Change runtime type -> GPU


In [3]:
# Install required packages
print("Installing dependencies...")

!pip install torch torchvision numpy pandas matplotlib seaborn tqdm scikit-learn pyyaml pillow scipy tensorboard -q

# Try to install MedMNIST (optional)
try:
    !pip install medmnist --no-deps -q
    !pip install fire -q
    print(" MedMNIST installed")
except:
    print("  MedMNIST not available - will skip those experiments")

print("\n Installation complete!")

Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 2.9 MB/s eta 0:00:00
 MedMNIST installed

 Installation complete!


In [8]:
# Mount Google Drive to save results persistently
from google.colab import drive
drive.mount('/content/drive')

# Create project directory in Drive
import os
project_dir = '/content/drive/MyDrive/warmup_study'
os.makedirs(project_dir, exist_ok=True)
print(f" Project directory: {project_dir}")

# Change to project directory
os.chdir(project_dir)
print(f" Working directory: {os.getcwd()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Project directory: /content/drive/MyDrive/warmup_study
 Working directory: /content/drive/MyDrive/warmup_study


In [5]:
# Upload the project ZIP file
from google.colab import files
import zipfile

print(" Please upload warmup_study_ULTIMATE.zip")
uploaded = files.upload()

# Extract
for filename in uploaded.keys():
    print(f"Extracting {filename}...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('.')
    print(f" Extracted to {os.getcwd()}")

# Move into the warmup_study directory
if os.path.exists('warmup_study'):
    os.chdir('warmup_study')
    print(f" Changed to: {os.getcwd()}")

# List files to verify
print("\n Project files:")
for f in sorted(os.listdir('.')):
    if f.endswith('.py'):
        print(f"   {f}")

 Please upload warmup_study_ULTIMATE.zip


Saving warmup_study_ULTIMATE.zip to warmup_study_ULTIMATE (6).zip
Extracting warmup_study_ULTIMATE (6).zip...
 Extracted to /content/drive/MyDrive/warmup_study
 Changed to: /content/drive/MyDrive/warmup_study/warmup_study

 Project files:
   FULL_CHECKPOINTING_CELLS.py
   JUPYTER_CHECKPOINTED_CELLS.py
   SIMPLE_CHECKPOINTED_WEEKS.py
   analysis.py
   config.py
   datasets.py
   full_checkpointing.py
   generate_visualizations.py
   interpretability.py
   models.py
   run_experiments.py
   run_experiments_checkpointed.py
   scheduler.py
   train.py
   utils.py


In [10]:
# Quick test to verify everything works
print("Running quick test (5 epochs, 100 examples/class)...\n")

from config import ExperimentConfig, DataConfig, ModelConfig, TrainingConfig
from train import train

config = ExperimentConfig(
    experiment_name='colab_test',
    data=DataConfig(name='cifar10', examples_per_class=100, data_dir='./data'),
    model=ModelConfig(num_classes=10),
    training=TrainingConfig(
        optimizer='sgd',
        warmup_epochs=1,
        epochs=5,
        batch_size=128,
        seed=42,
        device='cuda' if torch.cuda.is_available() else 'cpu',
        track_interpretability=False
    ),
    output_dir='./results'
)

summary = train(config)
print(f"\n Test complete! Accuracy: {summary['best_val_acc']:.2f}%")
print(f"Time: {summary['training_time_seconds']/60:.1f} minutes")

Running quick test (5 epochs, 100 examples/class)...

Using device: cuda

Loading cifar10 dataset...
Created CIFAR-10 subset with 1000 examples (100 per class)
Training samples: 1000
Validation samples: 10000
Training batches: 8

Creating resnet18 model...
Model parameters: 11,173,962

Starting training...
Warmup epochs: 1
Total epochs: 5
Optimizer: sgd
Initial LR: 0.1
Batch size: 128


Epoch 0: 100%|██████████| 8/8 [00:01<00:00,  7.74it/s, loss=2.41, acc=11.5]


Epoch   0/5 | Train Loss: 2.4107 | Train Acc: 11.50% | Val Loss: 2.3264 | Val Acc: 11.97% | LR: 0.000000


Epoch 1: 100%|██████████| 8/8 [00:00<00:00,  8.52it/s, loss=10, acc=11.8]


Epoch   1/5 | Train Loss: 10.0498 | Train Acc: 11.80% | Val Loss: 12101512.6899 | Val Acc: 10.00% | LR: 0.100000


...

Epoch   2/5 | Train Loss: 4.5674 | Train Acc: 10.70% | Val Loss: 4599.9066 | Val Acc: 11.32% | LR: 0.085355


Epoch 3: 100%|██████████| 8/8 [00:00<00:00,  8.26it/s, loss=2.88, acc=12]


Epoch   3/5 | Train Loss: 2.8828 | Train Acc: 12.00% | Val Loss: 29.8266 | Val Acc: 10.13% | LR: 0.050000


Epoch 4: 100%|██████████| 8/8 [00:00<00:00,  8.29it/s, loss=2.54, acc=7.5]


Epoch   4/5 | Train Loss: 2.5379 | Train Acc: 7.50% | Val Loss: 3.4822 | Val Acc: 11.96% | LR: 0.014645
Training complete in 16.11 seconds
Best validation accuracy: 11.97%
Final validation accuracy: 11.96%

Results saved to ./results/metrics/colab_test

 Test complete! Accuracy: 11.97%
Time: 0.3 minutes


In [13]:
# ====================================================================
# CELL 0: CLEAN START
# ====================================================================

import os
import shutil
from datetime import datetime

print("🧹 CLEAN START PREPARATION")
print("="*70)

# Create backup directory with timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
backup_dir = f'./backup_{timestamp}'
os.makedirs(backup_dir, exist_ok=True)

# Backup old results if they exist
backed_up = False
if os.path.exists('./results'):
    try:
        shutil.copytree('./results', f'{backup_dir}/results')
        print(f" Backed up results to {backup_dir}/results")
        backed_up = True
    except:
        print("  Results backup failed (might be empty)")

if os.path.exists('./experiment_checkpoints'):
    try:
        shutil.copytree('./experiment_checkpoints', f'{backup_dir}/experiment_checkpoints')
        print(f" Backed up checkpoints to {backup_dir}/experiment_checkpoints")
        backed_up = True
    except:
        print(" Checkpoint backup failed (might be empty)")

if backed_up:
    print(f"\nAll backups saved in: {backup_dir}")
else:
    print("\n No previous results to backup")

# Clean slate - remove old data
print("\n Cleaning old data...")

if os.path.exists('./experiment_checkpoints'):
    shutil.rmtree('./experiment_checkpoints')
    print(" Removed old experiment checkpoints")

if os.path.exists('./epoch_checkpoints'):
    shutil.rmtree('./epoch_checkpoints')
    print(" Removed old epoch checkpoints")

if os.path.exists('./results'):
    # Ask user
    response = input("\n  Remove old results folder? (yes/no): ")
    if response.lower() == 'yes':
        shutil.rmtree('./results')
        print("Removed old results")
    else:
        print("  Keeping old results (may cause confusion)")

print("\n" + "="*70)
print(" CLEAN START READY!")
print("="*70)
print(" All experiments will now run with:")
print("   • Batch size: 256")
print("   • Epochs: 85")
print("   • Fresh start from epoch 0")
print("\n Now run the main training code (Cell 6)!")
print("="*70)


🧹 CLEAN START PREPARATION
 Backed up results to ./backup_20251210_081834/results
 Backed up checkpoints to ./backup_20251210_081834/experiment_checkpoints

All backups saved in: ./backup_20251210_081834

 Cleaning old data...
 Removed old experiment checkpoints
 Removed old epoch checkpoints

  Remove old results folder? (yes/no): yes
Removed old results

 CLEAN START READY!
 All experiments will now run with:
   • Batch size: 256
   • Epochs: 85
   • Fresh start from epoch 0

 Now run the main training code (Cell 6)!


In [14]:
# ====================================================================
# WEEK 1 WITH EPOCH-LEVEL CHECKPOINTING - ENHANCED CONFIGURATION
# Optimized batch size and training duration for efficient execution
# ====================================================================

import json
import os
import torch
from datetime import datetime
from config import generate_experiment_grid
from train import set_seed
from datasets import get_dataloaders
from models import get_model
from scheduler import get_scheduler
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# ============================================================
#  TRAINING CONFIGURATION
# ============================================================
USE_ENHANCED_SETTINGS = True  # Set to False for standard settings

if USE_ENHANCED_SETTINGS:
    BATCH_SIZE = 256          # Larger batch for better GPU utilization
    TRAINING_EPOCHS = 85      # Sufficient for convergence
    CHECKPOINT_INTERVAL = 10  # Save every N epochs
    print(" Enhanced training configuration enabled")
    print(f"   • Batch size: {BATCH_SIZE}")
    print(f"   • Epochs: {TRAINING_EPOCHS}")
    print(f"   • Checkpoint interval: Every {CHECKPOINT_INTERVAL} epochs")
else:
    BATCH_SIZE = 128
    TRAINING_EPOCHS = 100
    CHECKPOINT_INTERVAL = 1
    print(" Standard training configuration")

# Check MedMNIST
try:
    import medmnist
    MEDMNIST_AVAILABLE = True
except ImportError:
    MEDMNIST_AVAILABLE = False

WEEK = 1

# Setup directories (in Google Drive for persistence!)
checkpoint_dir = './experiment_checkpoints'
epoch_checkpoint_dir = './epoch_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(epoch_checkpoint_dir, exist_ok=True)

exp_checkpoint_file = os.path.join(checkpoint_dir, f'week{WEEK}_checkpoint.json')

# Load experiment checkpoint
if os.path.exists(exp_checkpoint_file):
    with open(exp_checkpoint_file, 'r') as f:
        exp_checkpoint = json.load(f)
    completed_experiments = set(exp_checkpoint['completed_experiments'])
    results = exp_checkpoint['results']
    print(f" Resuming: {len(completed_experiments)} experiments done")
else:
    completed_experiments = set()
    results = []
    print(f" Starting Week {WEEK}")

# Generate experiments
all_exp = generate_experiment_grid()
if not MEDMNIST_AVAILABLE:
    all_exp = [e for e in all_exp if e.data.name != 'medmnist']

# Filter Week 1
experiments = [e for e in all_exp
              if e.data.name == 'cifar10'
              and e.training.optimizer == 'sgd'
              and e.training.seed == 42]

total = len(experiments)

# Calculate estimated time
avg_time_per_exp = 28 if USE_ENHANCED_SETTINGS else 40  # minutes
estimated_hours = (total - len(completed_experiments)) * avg_time_per_exp / 60

print(f"\n{'='*70}")
print(f" WEEK 1: {total} experiments")
print(f"Completed: {len(completed_experiments)}, Remaining: {total - len(completed_experiments)}")
print(f" Checkpoint interval: Every {CHECKPOINT_INTERVAL} epochs")
print(f"  Estimated: ~{estimated_hours:.1f} hours with GPU")
print(f"{'='*70}\n")

# Run experiments
import time
session_start = time.time()

for exp_idx, config in enumerate(experiments):
    exp_name = config.experiment_name

    if exp_name in completed_experiments:
        print(f" [{exp_idx+1}/{total}] Skip: {exp_name}")
        continue

    config.output_dir = './results'
    config.training.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # ============================================================
    #  APPLY TRAINING CONFIGURATION
    # ============================================================
    config.training.batch_size = BATCH_SIZE
    config.training.epochs = TRAINING_EPOCHS

    # Epoch checkpoint
    exp_epoch_checkpoint_dir = os.path.join(epoch_checkpoint_dir, exp_name)
    os.makedirs(exp_epoch_checkpoint_dir, exist_ok=True)
    epoch_checkpoint_file = os.path.join(exp_epoch_checkpoint_dir, 'checkpoint.pth')

    # Check for epoch checkpoint
    start_epoch = 0
    best_val_acc = 0.0
    all_metrics = {'train_loss': [], 'train_acc': [], 'val_loss': [],
                   'val_acc': [], 'learning_rate': [], 'epoch': []}

    if os.path.exists(epoch_checkpoint_file):
        print(f"\n{'='*70}")
        print(f" RESUMING [{exp_idx+1}/{total}] {exp_name}")
        checkpoint = torch.load(epoch_checkpoint_file)
        start_epoch = checkpoint['epoch'] + 1
        best_val_acc = checkpoint['best_val_acc']
        all_metrics = checkpoint['metrics']
        print(f"   From epoch {start_epoch}/{config.training.epochs}")
        print(f"   Best so far: {best_val_acc:.2f}%")
        print(f"{'='*70}")
        resume = True
    else:
        print(f"\n{'='*70}")
        print(f" [{len(completed_experiments)+1}/{total}] {exp_name}")
        print(f"   Size: {config.data.examples_per_class}/class | Warmup: {config.training.warmup_epochs}")
        print(f"   Batch: {BATCH_SIZE} | Epochs: {TRAINING_EPOCHS}")
        print(f"{'='*70}")
        resume = False

    try:
        # Setup
        set_seed(config.training.seed)
        device = torch.device(config.training.device)

        # Data
        if not resume:
            print(" Loading data...")
        train_loader, val_loader = get_dataloaders(
            dataset_name=config.data.name,
            data_dir=config.data.data_dir,
            examples_per_class=config.data.examples_per_class,
            batch_size=config.training.batch_size,
            num_workers=2,  # Reduced for Colab
            seed=config.training.seed
        )

        if not resume:
            print(f" Loaded {len(train_loader.dataset)} training samples")

        # Model
        model = get_model(config.model.name, num_classes=config.model.num_classes,
                         use_cifar_variant=True).to(device)

        # Optimizer
        if config.training.optimizer == 'sgd':
            optimizer = optim.SGD(model.parameters(), lr=config.training.lr,
                                momentum=config.training.momentum,
                                weight_decay=config.training.weight_decay)
        else:
            optimizer = optim.AdamW(model.parameters(), lr=config.training.lr,
                                  weight_decay=config.training.weight_decay)

        # Scheduler
        scheduler = get_scheduler(optimizer, config.training.warmup_epochs,
                                config.training.epochs, len(train_loader),
                                config.training.lr_schedule)

        # Load checkpoint state
        if resume:
            checkpoint = torch.load(epoch_checkpoint_file)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

        criterion = nn.CrossEntropyLoss()

        # Training loop
        exp_start_time = time.time()

        for epoch in range(start_epoch, config.training.epochs):
            # Train
            model.train()
            total_loss = 0.0
            correct = 0
            total_samples = 0

            pbar = tqdm(train_loader, desc=f' Epoch {epoch+1}/{config.training.epochs}',
                       leave=False)
            for data, target in pbar:
                if len(target.shape) > 1:
                    target = target.squeeze()
                data, target = data.to(device), target.to(device)

                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                pred = output.argmax(dim=1)
                correct += pred.eq(target).sum().item()
                total_samples += target.size(0)

                pbar.set_postfix({'loss': f'{total_loss/(pbar.n+1):.3f}',
                                'acc': f'{100.*correct/total_samples:.1f}%'})

            train_loss = total_loss / len(train_loader)
            train_acc = 100. * correct / total_samples

            # Validate
            model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for data, target in val_loader:
                    if len(target.shape) > 1:
                        target = target.squeeze()
                    data, target = data.to(device), target.to(device)
                    output = model(data)
                    val_loss += criterion(output, target).item()
                    val_correct += output.argmax(dim=1).eq(target).sum().item()
                    val_total += target.size(0)

            val_loss = val_loss / len(val_loader)
            val_acc = 100. * val_correct / val_total
            current_lr = optimizer.param_groups[0]['lr']

            # Update metrics
            all_metrics['train_loss'].append(train_loss)
            all_metrics['train_acc'].append(train_acc)
            all_metrics['val_loss'].append(val_loss)
            all_metrics['val_acc'].append(val_acc)
            all_metrics['learning_rate'].append(current_lr)
            all_metrics['epoch'].append(epoch)

            is_best = val_acc > best_val_acc
            if is_best:
                best_val_acc = val_acc

            # Print progress (every 10 epochs or if best)
            if (epoch + 1) % 10 == 0 or is_best:
                status = " NEW BEST!" if is_best else ""
                print(f"  Epoch {epoch+1:3d}/{config.training.epochs} | "
                      f"Train: {train_acc:5.1f}% | "
                      f"Val: {val_acc:5.1f}% {status}")

            # ============================================================
            #  SAVE CHECKPOINT AT SPECIFIED INTERVALS
            # ============================================================
            should_save = (
                (epoch + 1) % CHECKPOINT_INTERVAL == 0 or  # Regular interval
                is_best or                                  # New best model
                (epoch + 1) == config.training.epochs       # Final epoch
            )

            if should_save:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'best_val_acc': best_val_acc,
                    'metrics': all_metrics,
                }, epoch_checkpoint_file)

                if (epoch + 1) % 10 == 0:
                    print(f"      Checkpoint saved")

            scheduler.step()

        training_time = time.time() - exp_start_time

        print(f"\n{'='*70}")
        print(f" EXPERIMENT COMPLETE!")
        print(f"{'='*70}")
        print(f"    Best Accuracy: {best_val_acc:.2f}%")
        print(f"    Final Accuracy: {val_acc:.2f}%")
        print(f"     Time: {training_time/60:.1f} minutes")

        # Save results
        summary = {
            'experiment_name': exp_name,
            'dataset': config.data.name,
            'examples_per_class': config.data.examples_per_class,
            'warmup_epochs': config.training.warmup_epochs,
            'optimizer': config.training.optimizer,
            'seed': config.training.seed,
            'best_val_acc': best_val_acc,
            'final_val_acc': val_acc,
            'training_time_seconds': training_time,
            'total_epochs': config.training.epochs,
            'batch_size': config.training.batch_size,
        }
        results.append(summary)
        completed_experiments.add(exp_name)

        # Save experiment checkpoint
        with open(exp_checkpoint_file, 'w') as f:
            json.dump({
                'week': WEEK,
                'completed_experiments': list(completed_experiments),
                'results': results,
                'last_updated': datetime.now().isoformat(),
                'training_config': {
                    'batch_size': BATCH_SIZE,
                    'epochs': TRAINING_EPOCHS,
                    'enhanced_settings': USE_ENHANCED_SETTINGS
                }
            }, f, indent=2)

        # Clean up epoch checkpoint
        if os.path.exists(epoch_checkpoint_file):
            os.remove(epoch_checkpoint_file)
            print(f"    Cleaned up epoch checkpoint")

        progress_pct = len(completed_experiments) / total * 100
        print(f"    Overall Progress: {len(completed_experiments)}/{total} ({progress_pct:.1f}%)")
        print(f"{'='*70}")

    except KeyboardInterrupt:
        print(f"\n  INTERRUPTED BY USER!")
        print(f" Checkpoint saved at epoch {epoch+1}")
        print(f" Just re-run this cell to continue!")
        break

    except Exception as e:
        print(f"\n ERROR in experiment: {exp_name}")
        print(f"   Error: {str(e)}")
        print(f"     Skipping to next experiment...")
        import traceback
        traceback.print_exc()
        continue

# ============================================================
#  SESSION SUMMARY
# ============================================================
session_time = (time.time() - session_start) / 3600  # hours

print(f"\n{'='*70}")
print(f" SESSION SUMMARY")
print(f"{'='*70}")
print(f"  Session time: {session_time:.1f} hours")
print(f" Completed: {len(completed_experiments)}/{total} experiments")
print(f" Progress: {len(completed_experiments)/total*100:.1f}%")

if USE_ENHANCED_SETTINGS:
    print(f"\n Training Configuration:")
    print(f"   • Batch size: {BATCH_SIZE}")
    print(f"   • Epochs: {TRAINING_EPOCHS}")
    if len(completed_experiments) > 0:
        avg_time = session_time * 60 / len(completed_experiments)
        print(f"   • Avg time per exp: {avg_time:.1f} min")

if len(completed_experiments) < total:
    remaining = total - len(completed_experiments)
    est_time = remaining * avg_time_per_exp / 60
    print(f"\n Still need: {remaining} experiments")
    print(f"  Estimated time: ~{est_time:.1f} hours")
    print(f"\n TO CONTINUE:")
    print(f"   Just re-run this cell!")
    print(f"    Skips {len(completed_experiments)} completed experiments")
    print(f"     Resumes from experiment {len(completed_experiments)+1}")
else:
    print(f"\n WEEK {WEEK} COMPLETE! ")
    print(f" All {total} experiments finished!")
    print(f" Results saved in Google Drive")
    print(f"\n Next step: Change WEEK = 2 and run again!")

print(f"{'='*70}")

 Enhanced training configuration enabled
   • Batch size: 256
   • Epochs: 85
   • Checkpoint interval: Every 10 epochs
 Starting Week 1
Generated 155 experiment configurations

 WEEK 1: 25 experiments
Completed: 0, Remaining: 25
 Checkpoint interval: Every 10 epochs
  Estimated: ~11.7 hours with GPU


 [1/25] cifar10_5000epc_w0_sgd_s42
   Size: 5000/class | Warmup: 0
   Batch: 256 | Epochs: 85
 Loading data...
Created CIFAR-10 subset with 50000 examples (5000 per class)
 Loaded 50000 training samples


  Epoch   1/85 | Train:  25.1% | Val:  33.6%  NEW BEST!


  Epoch   2/85 | Train:  41.1% | Val:  47.1%  NEW BEST!


  Epoch   3/85 | Train:  49.9% | Val:  53.6%  NEW BEST!


  Epoch   4/85 | Train:  57.3% | Val:  59.5%  NEW BEST!


...

  Epoch  75/85 | Train: 100.0% | Val:  41.6%  NEW BEST!


  Epoch  79/85 | Train: 100.0% | Val:  41.6%  NEW BEST!


  Epoch  80/85 | Train: 100.0% | Val:  41.7%  NEW BEST!
      Checkpoint saved


  Epoch  81/85 | Train: 100.0% | Val:  41.8%  NEW BEST!


  Epoch  85/85 | Train: 100.0% | Val:  41.8%  NEW BEST!

 EXPERIMENT COMPLETE!
    Best Accuracy: 41.77%
    Final Accuracy: 41.77%
     Time: 4.4 minutes
    Cleaned up epoch checkpoint
    Overall Progress: 25/25 (100.0%)

 SESSION SUMMARY
  Session time: 8.0 hours
 Completed: 25/25 experiments
 Progress: 100.0%

 Training Configuration:
   • Batch size: 256
   • Epochs: 85
   • Avg time per exp: 19.1 min

 WEEK 1 COMPLETE! 
 All 25 experiments finished!
 Results saved in Google Drive

 Next step: Change WEEK = 2 and run again!


In [5]:
# ====================================================================
# WEEK 2: CIFAR-10 FULL GRID (SEEDS 123 & 456) - 50 EXPERIMENTS
# Enhanced configuration with batch 256 and 85 epochs
# ====================================================================

import json
from config import ExperimentConfig, DataConfig, ModelConfig, TrainingConfig
import os
import torch
from datetime import datetime
from config import generate_experiment_grid
from train import set_seed
from datasets import get_dataloaders
from models import get_model
from scheduler import get_scheduler
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import time

# ============================================================
#  TRAINING CONFIGURATION
# ============================================================
USE_ENHANCED_SETTINGS = True  # Set to False for standard settings

if USE_ENHANCED_SETTINGS:
    BATCH_SIZE = 256          # Larger batch for better GPU utilization
    TRAINING_EPOCHS = 85      # Sufficient for convergence
    CHECKPOINT_INTERVAL = 10  # Save every N epochs
    print("⚡ Enhanced training configuration enabled")
    print(f"   • Batch size: {BATCH_SIZE}")
    print(f"   • Epochs: {TRAINING_EPOCHS}")
    print(f"   • Checkpoint interval: Every {CHECKPOINT_INTERVAL} epochs")
else:
    BATCH_SIZE = 128
    TRAINING_EPOCHS = 100
    CHECKPOINT_INTERVAL = 1
    print(" Standard training configuration")

# Check MedMNIST
try:
    import medmnist
    MEDMNIST_AVAILABLE = True
except ImportError:
    MEDMNIST_AVAILABLE = False

# ============================================================
#  WEEK 2 CONFIGURATION
# ============================================================
WEEK = 2

print("\n" + "="*70)
print(" WEEK 2: CIFAR-10 STATISTICAL VALIDATION")
print("="*70)
print("Dataset: CIFAR-10")
print("Optimizer: SGD")
print("Seeds: 123 & 456 (for statistical reliability)")
print("Purpose: Add error bars and validate Week 1 findings")
print("="*70 + "\n")

# Setup directories (in Google Drive for persistence!)
checkpoint_dir = './experiment_checkpoints'
epoch_checkpoint_dir = './epoch_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(epoch_checkpoint_dir, exist_ok=True)

exp_checkpoint_file = os.path.join(checkpoint_dir, f'week{WEEK}_checkpoint.json')

# Load experiment checkpoint
if os.path.exists(exp_checkpoint_file):
    with open(exp_checkpoint_file, 'r') as f:
        exp_checkpoint = json.load(f)
    completed_experiments = set(exp_checkpoint['completed_experiments'])
    results = exp_checkpoint['results']
    print(f" Resuming Week 2: {len(completed_experiments)} experiments done")
else:
    completed_experiments = set()
    results = []
    print(f" Starting Week 2 from scratch")

# Generate experiments
print(" Generating experiment grid...")
all_exp = generate_experiment_grid()
if not MEDMNIST_AVAILABLE:
    all_exp = [e for e in all_exp if e.data.name != 'medmnist']
    if not MEDMNIST_AVAILABLE:
        print("  MedMNIST not available - continuing without it")

# ============================================================
#  FILTER WEEK 2: CIFAR-10, SGD, SEEDS 123 & 456
# ============================================================
experiments = [e for e in all_exp
              if e.data.name == 'cifar10'
              and e.training.optimizer == 'sgd'
              and e.training.seed in [123, 456]]

total = len(experiments)

# Verify correct count
assert total == 50, f"ERROR: Expected 50 experiments for Week 2, got {total}"
print(f" Generated {total} experiments for Week 2")

# Count by seed
seed_123 = [e for e in experiments if e.training.seed == 123]
seed_456 = [e for e in experiments if e.training.seed == 456]
print(f"   • Seed 123: {len(seed_123)} experiments")
print(f"   • Seed 456: {len(seed_456)} experiments")

# Calculate estimated time
avg_time_per_exp = 31 if USE_ENHANCED_SETTINGS else 45  # minutes
remaining = total - len(completed_experiments)
estimated_hours = remaining * avg_time_per_exp / 60

print(f"\n{'='*70}")
print(f" WEEK 2 EXECUTION PLAN")
print(f"{'='*70}")
print(f" Total experiments: {total}")
print(f" Completed: {len(completed_experiments)}")
print(f" Remaining: {remaining}")
print(f" Checkpoint interval: Every {CHECKPOINT_INTERVAL} epochs")
print(f"  Estimated time: ~{estimated_hours:.1f} hours")
print(f" Expected completion: ~{estimated_hours:.0f}h from now")
print(f"{'='*70}\n")

# ============================================================
#  RUN EXPERIMENTS
# ============================================================

session_start = time.time()

for exp_idx, config in enumerate(experiments):
    exp_name = config.experiment_name

    if exp_name in completed_experiments:
        print(f"  [{exp_idx+1}/{total}] Skip: {exp_name}")
        continue

    config.output_dir = './results'
    config.training.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Apply training configuration
    config.training.batch_size = BATCH_SIZE
    config.training.epochs = TRAINING_EPOCHS

    # Epoch checkpoint setup
    exp_epoch_checkpoint_dir = os.path.join(epoch_checkpoint_dir, exp_name)
    os.makedirs(exp_epoch_checkpoint_dir, exist_ok=True)
    epoch_checkpoint_file = os.path.join(exp_epoch_checkpoint_dir, 'checkpoint.pth')

    # Check for epoch checkpoint
    start_epoch = 0
    best_val_acc = 0.0
    all_metrics = {'train_loss': [], 'train_acc': [], 'val_loss': [],
                   'val_acc': [], 'learning_rate': [], 'epoch': []}

    if os.path.exists(epoch_checkpoint_file):
        print(f"\n{'='*70}")
        print(f" RESUMING [{exp_idx+1}/{total}] {exp_name}")
        checkpoint = torch.load(epoch_checkpoint_file, map_location='cpu')
        start_epoch = checkpoint['epoch'] + 1
        best_val_acc = checkpoint['best_val_acc']
        all_metrics = checkpoint['metrics']
        print(f"   Continuing from epoch {start_epoch}/{config.training.epochs}")
        print(f"   Best so far: {best_val_acc:.2f}%")
        print(f"{'='*70}")
        resume = True
    else:
        print(f"\n{'='*70}")
        print(f"[{len(completed_experiments)+1}/{total}] {exp_name}")
        print(f"{'='*70}")
        print(f"   Dataset: {config.data.name}")
        print(f"   Size: {config.data.examples_per_class} examples/class")
        print(f"   Warmup: {config.training.warmup_epochs} epochs")
        print(f"   Seed: {config.training.seed}")
        print(f"   Batch size: {BATCH_SIZE}")
        print(f"   Total epochs: {TRAINING_EPOCHS}")
        print(f"{'='*70}")
        resume = False

    try:
        # Setup
        set_seed(config.training.seed)
        device = torch.device(config.training.device)

        # Load data
        if not resume:
            print(" Loading data...")

        train_loader, val_loader = get_dataloaders(
            dataset_name=config.data.name,
            data_dir=config.data.data_dir,
            examples_per_class=config.data.examples_per_class,
            batch_size=config.training.batch_size,
            num_workers=2,
            seed=config.training.seed
        )

        if not resume:
            print(f" Training samples: {len(train_loader.dataset)}")
            print(f" Validation samples: {len(val_loader.dataset)}")

        # Create model
        model = get_model(
            config.model.name,
            num_classes=config.model.num_classes,
            use_cifar_variant=True
        ).to(device)

        # Optimizer
        if config.training.optimizer == 'sgd':
            optimizer = optim.SGD(
                model.parameters(),
                lr=config.training.lr,
                momentum=config.training.momentum,
                weight_decay=config.training.weight_decay
            )
        else:
            optimizer = optim.AdamW(
                model.parameters(),
                lr=config.training.lr,
                weight_decay=config.training.weight_decay
            )

        # Scheduler
        scheduler = get_scheduler(
            optimizer=optimizer,
            warmup_epochs=config.training.warmup_epochs,
            total_epochs=config.training.epochs,
            steps_per_epoch=len(train_loader),
            decay_schedule=config.training.lr_schedule
        )

        # Load checkpoint state if resuming
        if resume:
            checkpoint = torch.load(epoch_checkpoint_file, map_location=device)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

        criterion = nn.CrossEntropyLoss()

        # Training loop
        print(f"\n Starting training...")
        exp_start_time = time.time()

        for epoch in range(start_epoch, config.training.epochs):
            # Train one epoch
            model.train()
            total_loss = 0.0
            correct = 0
            total_samples = 0

            pbar = tqdm(
                train_loader,
                desc=f' Epoch {epoch+1}/{config.training.epochs}',
                leave=False
            )

            for data, target in pbar:
                if len(target.shape) > 1:
                    target = target.squeeze()

                data, target = data.to(device), target.to(device)

                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                pred = output.argmax(dim=1)
                correct += pred.eq(target).sum().item()
                total_samples += target.size(0)

                pbar.set_postfix({
                    'loss': f'{total_loss/(pbar.n+1):.3f}',
                    'acc': f'{100.*correct/total_samples:.1f}%'
                })

            train_loss = total_loss / len(train_loader)
            train_acc = 100. * correct / total_samples

            # Validate
            model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for data, target in val_loader:
                    if len(target.shape) > 1:
                        target = target.squeeze()
                    data, target = data.to(device), target.to(device)
                    output = model(data)
                    val_loss += criterion(output, target).item()
                    val_correct += output.argmax(dim=1).eq(target).sum().item()
                    val_total += target.size(0)

            val_loss = val_loss / len(val_loader)
            val_acc = 100. * val_correct / val_total
            current_lr = optimizer.param_groups[0]['lr']

            # Update metrics
            all_metrics['train_loss'].append(train_loss)
            all_metrics['train_acc'].append(train_acc)
            all_metrics['val_loss'].append(val_loss)
            all_metrics['val_acc'].append(val_acc)
            all_metrics['learning_rate'].append(current_lr)
            all_metrics['epoch'].append(epoch)

            # Check if best
            is_best = val_acc > best_val_acc
            if is_best:
                best_val_acc = val_acc

            # Print progress
            if (epoch + 1) % 10 == 0 or is_best:
                status = " NEW BEST!" if is_best else ""
                print(f"  Epoch {epoch+1:3d}/{config.training.epochs} | "
                      f"Train: {train_acc:5.1f}% | "
                      f"Val: {val_acc:5.1f}% | "
                      f"LR: {current_lr:.6f} {status}")

            # Save checkpoint
            should_save = (
                (epoch + 1) % CHECKPOINT_INTERVAL == 0 or
                is_best or
                (epoch + 1) == config.training.epochs
            )

            if should_save:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'best_val_acc': best_val_acc,
                    'metrics': all_metrics,
                }, epoch_checkpoint_file)

                if (epoch + 1) % 10 == 0:
                    print(f"      Checkpoint saved")

            scheduler.step()

        training_time = time.time() - exp_start_time

        # Experiment complete
        print(f"\n{'='*70}")
        print(f" EXPERIMENT COMPLETE!")
        print(f"{'='*70}")
        print(f"    Best Accuracy: {best_val_acc:.2f}%")
        print(f"    Final Accuracy: {val_acc:.2f}%")
        print(f"     Training Time: {training_time/60:.1f} minutes")

        # Save results
        summary = {
            'experiment_name': exp_name,
            'dataset': config.data.name,
            'examples_per_class': config.data.examples_per_class,
            'warmup_epochs': config.training.warmup_epochs,
            'optimizer': config.training.optimizer,
            'seed': config.training.seed,
            'best_val_acc': best_val_acc,
            'final_val_acc': val_acc,
            'training_time_seconds': training_time,
            'total_epochs': config.training.epochs,
            'batch_size': config.training.batch_size,
        }

        results.append(summary)
        completed_experiments.add(exp_name)

        # Save experiment checkpoint
        with open(exp_checkpoint_file, 'w') as f:
            json.dump({
                'week': WEEK,
                'completed_experiments': list(completed_experiments),
                'results': results,
                'last_updated': datetime.now().isoformat(),
                'training_config': {
                    'batch_size': BATCH_SIZE,
                    'epochs': TRAINING_EPOCHS,
                    'enhanced_settings': USE_ENHANCED_SETTINGS
                }
            }, f, indent=2)

        # Clean up epoch checkpoint
        if os.path.exists(epoch_checkpoint_file):
            os.remove(epoch_checkpoint_file)
            print(f"    Cleaned up epoch checkpoint")

        progress_pct = len(completed_experiments) / total * 100

        # Calculate ETA
        elapsed_time = (time.time() - session_start) / 3600  # hours
        if len(completed_experiments) > 0:
            avg_time_actual = elapsed_time / len(completed_experiments)
            remaining_time = avg_time_actual * (total - len(completed_experiments))
        else:
            remaining_time = estimated_hours

        print(f"    Progress: {len(completed_experiments)}/{total} ({progress_pct:.1f}%)")
        print(f"     ETA: ~{remaining_time:.1f} hours remaining")
        print(f"{'='*70}")

    except KeyboardInterrupt:
        print(f"\n  INTERRUPTED BY USER!")
        print(f" Checkpoint saved at epoch {epoch+1}")
        print(f" Re-run this cell to continue from where you stopped!")
        break

    except Exception as e:
        print(f"\n ERROR in experiment: {exp_name}")
        print(f"   Error: {str(e)}")
        print(f"     Skipping to next experiment...")
        import traceback
        traceback.print_exc()
        continue

# ============================================================
#  WEEK 2 SESSION SUMMARY
# ============================================================

session_time = (time.time() - session_start) / 3600  # hours

print(f"\n{'='*70}")
print(f" WEEK 2 SESSION SUMMARY")
print(f"{'='*70}")
print(f"  Session time: {session_time:.1f} hours")
print(f" Completed: {len(completed_experiments)}/{total} experiments")
print(f" Progress: {len(completed_experiments)/total*100:.1f}%")

if USE_ENHANCED_SETTINGS:
    print(f"\n Training Configuration:")
    print(f"   • Batch size: {BATCH_SIZE}")
    print(f"   • Epochs: {TRAINING_EPOCHS}")
    if len(completed_experiments) > 0:
        avg_time = session_time * 60 / len(completed_experiments)
        print(f"   • Avg time per experiment: {avg_time:.1f} minutes")

# Show results by seed if available
if results:
    print(f"\n Results Summary:")
    seed_123_results = [r for r in results if r.get('seed') == 123]
    seed_456_results = [r for r in results if r.get('seed') == 456]

    if seed_123_results:
        avg_acc_123 = sum(r['best_val_acc'] for r in seed_123_results) / len(seed_123_results)
        print(f"   • Seed 123: {len(seed_123_results)} exp, avg acc: {avg_acc_123:.2f}%")

    if seed_456_results:
        avg_acc_456 = sum(r['best_val_acc'] for r in seed_456_results) / len(seed_456_results)
        print(f"   • Seed 456: {len(seed_456_results)} exp, avg acc: {avg_acc_456:.2f}%")

if len(completed_experiments) < total:
    remaining = total - len(completed_experiments)
    est_time = remaining * avg_time_per_exp / 60
    print(f"\n Still need: {remaining} experiments")
    print(f"  Estimated time: ~{est_time:.1f} hours")
    print(f"\n TO CONTINUE:")
    print(f"   Just re-run this cell!")
    print(f"    Auto-skips {len(completed_experiments)} completed experiments")
    print(f"     Resumes from experiment {len(completed_experiments)+1}")
else:
    print(f"\n WEEK 2 COMPLETE! ")
    print(f" All {total} experiments finished!")
    print(f" Results saved in Google Drive")
    print(f"\n Statistical Analysis Ready:")
    print(f"   • 3 seeds total (42, 123, 456) for CIFAR-10")
    print(f"   • Can now calculate mean ± std deviation")
    print(f"   • Error bars for all plots")
    print(f"   • Statistical significance testing")
    print(f"\n Next: Run Week 3 for cross-dataset validation!")

print(f"{'='*70}")

⚡ Enhanced training configuration enabled
   • Batch size: 256
   • Epochs: 85
   • Checkpoint interval: Every 10 epochs

 WEEK 2: CIFAR-10 STATISTICAL VALIDATION
Dataset: CIFAR-10
Optimizer: SGD
Seeds: 123 & 456 (for statistical reliability)
Purpose: Add error bars and validate Week 1 findings

 Resuming Week 2: 9 experiments done
 Generating experiment grid...
Generated 155 experiment configurations
 Generated 50 experiments for Week 2
   • Seed 123: 25 experiments
   • Seed 456: 25 experiments

 WEEK 2 EXECUTION PLAN
 Total experiments: 50
 Completed: 9
 Remaining: 41
 Checkpoint interval: Every 10 epochs
  Estimated time: ~21.2 hours
 Expected completion: ~21h from now

  [1/50] Skip: cifar10_5000epc_w0_sgd_s123
  [2/50] Skip: cifar10_5000epc_w0_sgd_s456
  [3/50] Skip: cifar10_5000epc_w1_sgd_s123
  [4/50] Skip: cifar10_5000epc_w1_sgd_s456
  [5/50] Skip: cifar10_5000epc_w5_sgd_s123
  [6/50] Skip: cifar10_5000epc_w5_sgd_s456
  [7/50] Skip: cifar10_5000epc_w10_sgd_s123
  [8/50] Skip: 

  Epoch  60/85 | Train:  97.3% | Val:  90.8% | LR: 0.039211  NEW BEST!
      Checkpoint saved


  Epoch  62/85 | Train:  97.6% | Val:  91.6% | LR: 0.034549  NEW BEST!


  Epoch  63/85 | Train:  98.0% | Val:  91.7% | LR: 0.032270  NEW BEST!


  Epoch  64/85 | Train:  98.3% | Val:  92.0% | LR: 0.030032  NEW BEST!


...

  Epoch  80/85 | Train: 100.0% | Val:  40.8% | LR: 0.002088  NEW BEST!
      Checkpoint saved


  Epoch  82/85 | Train: 100.0% | Val:  40.8% | LR: 0.000931  NEW BEST!


  Epoch  83/85 | Train:  99.8% | Val:  40.9% | LR: 0.000525  NEW BEST!


  Epoch  84/85 | Train: 100.0% | Val:  40.9% | LR: 0.000233  NEW BEST!



 EXPERIMENT COMPLETE!
    Best Accuracy: 40.88%
    Final Accuracy: 40.86%
     Training Time: 2.5 minutes
    Cleaned up epoch checkpoint
    Progress: 50/50 (100.0%)
     ETA: ~0.0 hours remaining

 WEEK 2 SESSION SUMMARY
  Session time: 2.5 hours
 Completed: 50/50 experiments
 Progress: 100.0%

 Training Configuration:
   • Batch size: 256
   • Epochs: 85
   • Avg time per experiment: 3.0 minutes

 Results Summary:
   • Seed 123: 25 exp, avg acc: 67.10%
   • Seed 456: 25 exp, avg acc: 65.41%

 WEEK 2 COMPLETE! 
 All 50 experiments finished!
 Results saved in Google Drive

 Statistical Analysis Ready:
   • 3 seeds total (42, 123, 456) for CIFAR-10
   • Can now calculate mean ± std deviation
   • Error bars for all plots
   • Statistical significance testing

 Next: Run Week 3 for cross-dataset validation!


In [5]:
# ====================================================================
# WEEK 3: CROSS-DATASET VALIDATION - 80 EXPERIMENTS
# CIFAR-100 (25) + MedMNIST (25) + AdamW (30)
# Enhanced configuration with batch 256 and 85 epochs
# ====================================================================

import json
import os
import torch
from datetime import datetime
from config import generate_experiment_grid
from train import set_seed
from datasets import get_dataloaders
from models import get_model
from scheduler import get_scheduler
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import time

# ============================================================
# TRAINING CONFIGURATION
# ============================================================
USE_ENHANCED_SETTINGS = True  # Set to False for standard settings

if USE_ENHANCED_SETTINGS:
    BATCH_SIZE = 256          # Larger batch for better GPU utilization
    TRAINING_EPOCHS = 85      # Sufficient for convergence
    CHECKPOINT_INTERVAL = 10  # Save every N epochs
    print("Enhanced training configuration enabled")
    print(f"   Batch size: {BATCH_SIZE}")
    print(f"   Epochs: {TRAINING_EPOCHS}")
    print(f"   Checkpoint interval: Every {CHECKPOINT_INTERVAL} epochs")
else:
    BATCH_SIZE = 128
    TRAINING_EPOCHS = 100
    CHECKPOINT_INTERVAL = 1
    print("Standard training configuration")

# Check MedMNIST
try:
    import medmnist
    MEDMNIST_AVAILABLE = True
    print("MedMNIST available")
except ImportError:
    MEDMNIST_AVAILABLE = False
    print("WARNING: MedMNIST not available - will skip those 25 experiments")

# ============================================================
# WEEK 3 CONFIGURATION
# ============================================================
WEEK = 3

print("\n" + "="*70)
print("WEEK 3: CROSS-DATASET VALIDATION")
print("="*70)
print("Groups:")
print("  - CIFAR-100 + SGD (25 exp) - 100-class validation")
print("  - MedMNIST + SGD (25 exp)  - Medical domain validation")
print("  - AdamW (30 exp)           - Optimizer comparison")
print("Total: 80 experiments")
print("Purpose: Test H3 (generalization) & H4 (AdamW sensitivity)")
print("="*70 + "\n")

# Setup directories
checkpoint_dir = './experiment_checkpoints'
epoch_checkpoint_dir = './epoch_checkpoints'
results_base_dir = './results'

os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(epoch_checkpoint_dir, exist_ok=True)
os.makedirs(results_base_dir, exist_ok=True)
os.makedirs(os.path.join(results_base_dir, 'metrics'), exist_ok=True)
os.makedirs(os.path.join(results_base_dir, 'models'), exist_ok=True)

exp_checkpoint_file = os.path.join(checkpoint_dir, f'week{WEEK}_checkpoint.json')

# Load experiment checkpoint
if os.path.exists(exp_checkpoint_file):
    with open(exp_checkpoint_file, 'r') as f:
        exp_checkpoint = json.load(f)
    completed_experiments = set(exp_checkpoint['completed_experiments'])
    results = exp_checkpoint['results']
    print(f"Resuming Week 3: {len(completed_experiments)} experiments done")
else:
    completed_experiments = set()
    results = []
    print(f"Starting Week 3 from scratch")

# Generate experiments
print("Generating experiment grid...")
all_exp = generate_experiment_grid()

# ============================================================
# FILTER WEEK 3: CIFAR-100 + MedMNIST + AdamW
# ============================================================
experiments = [e for e in all_exp
              if e.data.name in ['cifar100', 'medmnist']
              or e.training.optimizer == 'adamw']

# If MedMNIST not available, filter it out
if not MEDMNIST_AVAILABLE:
    experiments = [e for e in experiments if e.data.name != 'medmnist']
    print("Filtered out MedMNIST experiments (not available)")

total = len(experiments)

# Expected: 80 with MedMNIST, 55 without MedMNIST
expected = 80 if MEDMNIST_AVAILABLE else 55
if total != expected:
    print(f"WARNING: Expected {expected} experiments, got {total}")
else:
    print(f"Generated {total} experiments for Week 3")

# Count by group
cifar100_sgd = [e for e in experiments if e.data.name == 'cifar100' and e.training.optimizer == 'sgd']
medmnist_sgd = [e for e in experiments if e.data.name == 'medmnist' and e.training.optimizer == 'sgd']
adamw_all = [e for e in experiments if e.training.optimizer == 'adamw']

print(f"\nWeek 3 Breakdown:")
print(f"   CIFAR-100 + SGD: {len(cifar100_sgd)} experiments")
if MEDMNIST_AVAILABLE:
    print(f"   MedMNIST + SGD:  {len(medmnist_sgd)} experiments")
else:
    print(f"   MedMNIST + SGD:  0 experiments (not available)")
print(f"   AdamW (all):     {len(adamw_all)} experiments")

# Calculate estimated time
avg_time_per_exp = 31 if USE_ENHANCED_SETTINGS else 45  # minutes
remaining = total - len(completed_experiments)
estimated_hours = remaining * avg_time_per_exp / 60

print(f"\n{'='*70}")
print(f"WEEK 3 EXECUTION PLAN")
print(f"{'='*70}")
print(f"Total experiments: {total}")
print(f"Completed: {len(completed_experiments)}")
print(f"Remaining: {remaining}")
print(f"Checkpoint interval: Every {CHECKPOINT_INTERVAL} epochs")
print(f"Estimated time: ~{estimated_hours:.1f} hours")
print(f"Expected completion: ~{estimated_hours:.0f}h from now")
if MEDMNIST_AVAILABLE:
    print(f"Includes medical imaging experiments")
print(f"{'='*70}\n")

# ============================================================
# RUN EXPERIMENTS
# ============================================================

session_start = time.time()
experiments_this_session = 0

for exp_idx, config in enumerate(experiments):
    exp_name = config.experiment_name

    if exp_name in completed_experiments:
        print(f"[{exp_idx+1}/{total}] Skip: {exp_name}")
        continue

    # Apply configuration
    config.output_dir = results_base_dir
    config.training.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    config.training.batch_size = BATCH_SIZE
    config.training.epochs = TRAINING_EPOCHS

    # Epoch checkpoint setup
    exp_epoch_checkpoint_dir = os.path.join(epoch_checkpoint_dir, exp_name)
    os.makedirs(exp_epoch_checkpoint_dir, exist_ok=True)
    epoch_checkpoint_file = os.path.join(exp_epoch_checkpoint_dir, 'checkpoint.pth')

    # Check for epoch checkpoint
    start_epoch = 0
    best_val_acc = 0.0
    all_metrics = {'train_loss': [], 'train_acc': [], 'val_loss': [],
                   'val_acc': [], 'learning_rate': [], 'epoch': []}

    if os.path.exists(epoch_checkpoint_file):
        print(f"\n{'='*70}")
        print(f"RESUMING [{exp_idx+1}/{total}] {exp_name}")
        checkpoint = torch.load(epoch_checkpoint_file, map_location='cpu')
        start_epoch = checkpoint['epoch'] + 1
        best_val_acc = checkpoint['best_val_acc']
        all_metrics = checkpoint['metrics']
        print(f"   Continuing from epoch {start_epoch}/{config.training.epochs}")
        print(f"   Best so far: {best_val_acc:.2f}%")
        print(f"{'='*70}")
        resume = True
    else:
        print(f"\n{'='*70}")
        print(f"STARTING [{len(completed_experiments)+1}/{total}] {exp_name}")
        print(f"{'='*70}")
        print(f"   Dataset: {config.data.name}")
        print(f"   Size: {config.data.examples_per_class} examples/class")
        print(f"   Warmup: {config.training.warmup_epochs} epochs")
        print(f"   Optimizer: {config.training.optimizer}")
        print(f"   Seed: {config.training.seed}")
        print(f"   Batch size: {BATCH_SIZE}")
        print(f"   Total epochs: {TRAINING_EPOCHS}")
        print(f"{'='*70}")
        resume = False

    try:
        # Setup
        set_seed(config.training.seed)
        device = torch.device(config.training.device)

        # Load data
        if not resume:
            print("Loading data...")

        train_loader, val_loader = get_dataloaders(
            dataset_name=config.data.name,
            data_dir=config.data.data_dir,
            examples_per_class=config.data.examples_per_class,
            batch_size=config.training.batch_size,
            num_workers=2,
            seed=config.training.seed
        )

        if not resume:
            print(f"Training samples: {len(train_loader.dataset)}")
            print(f"Validation samples: {len(val_loader.dataset)}")

        # Create model (note: num_classes may differ!)
        model = get_model(
            config.model.name,
            num_classes=config.model.num_classes,
            use_cifar_variant=True
        ).to(device)

        if not resume:
            total_params = sum(p.numel() for p in model.parameters())
            print(f"Model parameters: {total_params:,}")

        # Optimizer
        if config.training.optimizer == 'sgd':
            optimizer = optim.SGD(
                model.parameters(),
                lr=config.training.lr,
                momentum=config.training.momentum,
                weight_decay=config.training.weight_decay
            )
        else:  # AdamW
            optimizer = optim.AdamW(
                model.parameters(),
                lr=config.training.lr,
                weight_decay=config.training.weight_decay
            )

        # Scheduler
        scheduler = get_scheduler(
            optimizer=optimizer,
            warmup_epochs=config.training.warmup_epochs,
            total_epochs=config.training.epochs,
            steps_per_epoch=len(train_loader),
            decay_schedule=config.training.lr_schedule
        )

        # Load checkpoint state if resuming
        if resume:
            checkpoint = torch.load(epoch_checkpoint_file, map_location=device)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

        criterion = nn.CrossEntropyLoss()

        # Training loop
        print(f"\nStarting training...")
        exp_start_time = time.time()

        for epoch in range(start_epoch, config.training.epochs):
            # Train one epoch
            model.train()
            total_loss = 0.0
            correct = 0
            total_samples = 0

            pbar = tqdm(
                train_loader,
                desc=f'Epoch {epoch+1}/{config.training.epochs}',
                leave=False
            )

            for data, target in pbar:
                if len(target.shape) > 1:
                    target = target.squeeze()

                data, target = data.to(device), target.to(device)

                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                pred = output.argmax(dim=1)
                correct += pred.eq(target).sum().item()
                total_samples += target.size(0)

                pbar.set_postfix({
                    'loss': f'{total_loss/(pbar.n+1):.3f}',
                    'acc': f'{100.*correct/total_samples:.1f}%'
                })

            train_loss = total_loss / len(train_loader)
            train_acc = 100. * correct / total_samples

            # Validate
            model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for data, target in val_loader:
                    if len(target.shape) > 1:
                        target = target.squeeze()
                    data, target = data.to(device), target.to(device)
                    output = model(data)
                    val_loss += criterion(output, target).item()
                    val_correct += output.argmax(dim=1).eq(target).sum().item()
                    val_total += target.size(0)

            val_loss = val_loss / len(val_loader)
            val_acc = 100. * val_correct / val_total
            current_lr = optimizer.param_groups[0]['lr']

            # Update metrics
            all_metrics['train_loss'].append(train_loss)
            all_metrics['train_acc'].append(train_acc)
            all_metrics['val_loss'].append(val_loss)
            all_metrics['val_acc'].append(val_acc)
            all_metrics['learning_rate'].append(current_lr)
            all_metrics['epoch'].append(epoch)

            # Check if best
            is_best = val_acc > best_val_acc
            if is_best:
                best_val_acc = val_acc

            # Print progress
            if (epoch + 1) % 10 == 0 or is_best:
                status = "NEW BEST" if is_best else ""
                print(f"  Epoch {epoch+1:3d}/{config.training.epochs} | "
                      f"Train: {train_acc:5.1f}% | "
                      f"Val: {val_acc:5.1f}% | "
                      f"LR: {current_lr:.6f} {status}")

            # Save checkpoint
            should_save = (
                (epoch + 1) % CHECKPOINT_INTERVAL == 0 or
                is_best or
                (epoch + 1) == config.training.epochs
            )

            if should_save:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'best_val_acc': best_val_acc,
                    'metrics': all_metrics,
                }, epoch_checkpoint_file)

                if (epoch + 1) % 10 == 0:
                    print(f"     Checkpoint saved")

            scheduler.step()

        training_time = time.time() - exp_start_time

        # ============================================================
        # SAVE RESULTS EXPLICITLY TO DISK
        # ============================================================

        # Create experiment directories
        metrics_dir = os.path.join(results_base_dir, 'metrics', exp_name)
        models_dir = os.path.join(results_base_dir, 'models', exp_name)
        os.makedirs(metrics_dir, exist_ok=True)
        os.makedirs(models_dir, exist_ok=True)

        # Save metrics
        metrics_file = os.path.join(metrics_dir, 'metrics.json')
        with open(metrics_file, 'w') as f:
            json.dump(all_metrics, f, indent=2)

        # Save summary
        summary = {
            'experiment_name': exp_name,
            'dataset': config.data.name,
            'examples_per_class': config.data.examples_per_class,
            'warmup_epochs': config.training.warmup_epochs,
            'optimizer': config.training.optimizer,
            'seed': config.training.seed,
            'best_val_acc': best_val_acc,
            'final_val_acc': val_acc,
            'training_time_seconds': training_time,
            'total_epochs': config.training.epochs,
            'batch_size': config.training.batch_size,
        }

        summary_file = os.path.join(metrics_dir, 'summary.json')
        with open(summary_file, 'w') as f:
            json.dump(summary, f, indent=2)

        # Save model
        model_file = os.path.join(models_dir, 'best_model.pth')
        torch.save({
            'model_state_dict': model.state_dict(),
            'best_val_acc': best_val_acc,
            'config': summary
        }, model_file)

        # Experiment complete
        print(f"\n{'='*70}")
        print(f"EXPERIMENT COMPLETE")
        print(f"{'='*70}")
        print(f"   Best Accuracy: {best_val_acc:.2f}%")
        print(f"   Final Accuracy: {val_acc:.2f}%")
        print(f"   Training Time: {training_time/60:.1f} minutes")
        print(f"   Saved to: {metrics_dir}")

        # Update tracking
        results.append(summary)
        completed_experiments.add(exp_name)
        experiments_this_session += 1

        # Save experiment checkpoint
        with open(exp_checkpoint_file, 'w') as f:
            json.dump({
                'week': WEEK,
                'completed_experiments': list(completed_experiments),
                'results': results,
                'last_updated': datetime.now().isoformat(),
                'training_config': {
                    'batch_size': BATCH_SIZE,
                    'epochs': TRAINING_EPOCHS,
                    'enhanced_settings': USE_ENHANCED_SETTINGS
                }
            }, f, indent=2)

        # Clean up epoch checkpoint
        if os.path.exists(epoch_checkpoint_file):
            os.remove(epoch_checkpoint_file)
            print(f"   Cleaned up epoch checkpoint")

        progress_pct = len(completed_experiments) / total * 100

        # Calculate ETA
        elapsed_time = (time.time() - session_start) / 3600  # hours
        if experiments_this_session > 0:
            avg_time_actual = elapsed_time / experiments_this_session
            remaining_exp = total - len(completed_experiments)
            remaining_time = avg_time_actual * remaining_exp
        else:
            remaining_time = estimated_hours

        print(f"   Progress: {len(completed_experiments)}/{total} ({progress_pct:.1f}%)")
        print(f"   ETA: ~{remaining_time:.1f} hours remaining")
        print(f"{'='*70}")

    except KeyboardInterrupt:
        print(f"\nINTERRUPTED BY USER")
        print(f"Checkpoint saved at epoch {epoch+1}")
        print(f"Re-run this cell to continue from where you stopped")
        break

    except Exception as e:
        print(f"\nERROR in experiment: {exp_name}")
        print(f"   Error: {str(e)}")
        print(f"   Skipping to next experiment...")
        import traceback
        traceback.print_exc()
        continue

# ============================================================
# WEEK 3 SESSION SUMMARY
# ============================================================

session_time = (time.time() - session_start) / 3600  # hours

print(f"\n{'='*70}")
print(f"WEEK 3 SESSION SUMMARY")
print(f"{'='*70}")
print(f"Session time: {session_time:.1f} hours")
print(f"Experiments this session: {experiments_this_session}")
print(f"Total completed: {len(completed_experiments)}/{total} experiments")
print(f"Progress: {len(completed_experiments)/total*100:.1f}%")

if USE_ENHANCED_SETTINGS:
    print(f"\nTraining Configuration:")
    print(f"   Batch size: {BATCH_SIZE}")
    print(f"   Epochs: {TRAINING_EPOCHS}")
    if experiments_this_session > 0:
        avg_time = session_time * 60 / experiments_this_session
        print(f"   Avg time per experiment: {avg_time:.1f} minutes")

# Show results by dataset if available
if results:
    print(f"\nResults Summary by Dataset:")

    cifar100_results = [r for r in results if r.get('dataset') == 'cifar100']
    if cifar100_results:
        avg_acc_c100 = sum(r['best_val_acc'] for r in cifar100_results) / len(cifar100_results)
        print(f"   CIFAR-100: {len(cifar100_results)} exp, avg acc: {avg_acc_c100:.2f}%")

    if MEDMNIST_AVAILABLE:
        medmnist_results = [r for r in results if r.get('dataset') == 'medmnist']
        if medmnist_results:
            avg_acc_med = sum(r['best_val_acc'] for r in medmnist_results) / len(medmnist_results)
            print(f"   MedMNIST:  {len(medmnist_results)} exp, avg acc: {avg_acc_med:.2f}%")

    # Show by optimizer
    print(f"\nResults by Optimizer:")
    sgd_results = [r for r in results if r.get('optimizer') == 'sgd']
    adamw_results = [r for r in results if r.get('optimizer') == 'adamw']

    if sgd_results:
        avg_acc_sgd = sum(r['best_val_acc'] for r in sgd_results) / len(sgd_results)
        print(f"   SGD:   {len(sgd_results)} exp, avg acc: {avg_acc_sgd:.2f}%")

    if adamw_results:
        avg_acc_adamw = sum(r['best_val_acc'] for r in adamw_results) / len(adamw_results)
        print(f"   AdamW: {len(adamw_results)} exp, avg acc: {avg_acc_adamw:.2f}%")

if len(completed_experiments) < total:
    remaining = total - len(completed_experiments)
    est_time = remaining * avg_time_per_exp / 60
    print(f"\nStill need: {remaining} experiments")
    print(f"Estimated time: ~{est_time:.1f} hours")
    print(f"\nTO CONTINUE:")
    print(f"   Just re-run this cell")
    print(f"   Auto-skips {len(completed_experiments)} completed experiments")
    print(f"   Resumes from experiment {len(completed_experiments)+1}")
else:
    print(f"\nWEEK 3 COMPLETE")
    print(f"All {total} experiments finished")
    print(f"Results saved in Google Drive")
    print(f"\nENTIRE PROJECT COMPLETE")
    print(f"   Week 1: 25 experiments")
    print(f"   Week 2: 50 experiments")
    print(f"   Week 3: {total} experiments")
    print(f"   TOTAL: {25 + 50 + total} experiments")
    print(f"\nNext steps:")
    print(f"   - Download results")
    print(f"   - Run analysis")
    print(f"   - Create visualizations")
    print(f"   - Write your report")

print(f"{'='*70}")

Enhanced training configuration enabled
   Batch size: 256
   Epochs: 85
   Checkpoint interval: Every 10 epochs
MedMNIST available

WEEK 3: CROSS-DATASET VALIDATION
Groups:
  - CIFAR-100 + SGD (25 exp) - 100-class validation
  - MedMNIST + SGD (25 exp)  - Medical domain validation
  - AdamW (30 exp)           - Optimizer comparison
Total: 80 experiments
Purpose: Test H3 (generalization) & H4 (AdamW sensitivity)

Resuming Week 3: 65 experiments done
Generating experiment grid...
Generated 155 experiment configurations
Generated 80 experiments for Week 3

Week 3 Breakdown:
   CIFAR-100 + SGD: 25 experiments
   MedMNIST + SGD:  25 experiments
   AdamW (all):     30 experiments

WEEK 3 EXECUTION PLAN
Total experiments: 80
Completed: 65
Remaining: 15
Checkpoint interval: Every 10 epochs
Estimated time: ~7.8 hours
Expected completion: ~8h from now
Includes medical imaging experiments

[1/80] Skip: cifar100_5000epc_w0_sgd_s42
[2/80] Skip: cifar100_5000epc_w1_sgd_s42
[3/80] Skip: cifar100_500


EXPERIMENT COMPLETE
   Best Accuracy: 72.72%
   Final Accuracy: 72.50%
   Training Time: 1.0 minutes
   Saved to: ./results/metrics/cifar100_500epc_w5_adamw_s42
   Cleaned up epoch checkpoint
   Progress: 66/80 (82.5%)
   ETA: ~0.3 hours remaining

STARTING [67/80] cifar10_500epc_w20_adamw_s42
   Dataset: cifar10
   Size: 500 examples/class
   Warmup: 20 epochs
   Optimizer: adamw
   Seed: 42
   Batch size: 256
   Total epochs: 85
Loading data...
Created CIFAR-10 subset with 5000 examples (500 per class)
Training samples: 5000
Validation samples: 10000
Model parameters: 11,173,962

Starting training...


  Epoch   1/85 | Train:  11.0% | Val:  12.1% | LR: 0.000000 NEW BEST


  Epoch   2/85 | Train:  20.1% | Val:  23.6% | LR: 0.000050 NEW BEST


  Epoch   3/85 | Train:  30.3% | Val:  30.7% | LR: 0.000100 NEW BEST


...

  Epoch   7/85 | Train:  41.2% | Val:  39.7% | LR: 0.000300 NEW BEST


  Epoch   8/85 | Train:  45.6% | Val:  44.7% | LR: 0.000350 NEW BEST


  Epoch  10/85 | Train:  52.7% | Val:  49.2% | LR: 0.000450 NEW BEST
     Checkpoint saved


  Epoch  12/85 | Train:  58.0% | Val:  51.3% | LR: 0.000550 NEW BEST



INTERRUPTED BY USER
Checkpoint saved at epoch 13
Re-run this cell to continue from where you stopped

WEEK 3 SESSION SUMMARY
Session time: 0.1 hours
Experiments this session: 2
Total completed: 67/80 experiments
Progress: 83.8%

Training Configuration:
   Batch size: 256
   Epochs: 85
   Avg time per experiment: 4.0 minutes

Results Summary by Dataset:
   CIFAR-100: 33 exp, avg acc: 65.01%
   MedMNIST:  25 exp, avg acc: 78.62%

Results by Optimizer:
   SGD:   50 exp, avg acc: 70.60%
   AdamW: 17 exp, avg acc: 78.63%

Still need: 13 experiments
Estimated time: ~6.7 hours

TO CONTINUE:
   Just re-run this cell
   Auto-skips 67 completed experiments
   Resumes from experiment 68


##Later part of it was done in warmup_2.ipynb

In [7]:
import shutil, os
from google.colab import files

os.chdir('/content/drive/MyDrive/warmup_study/warmup_study')
shutil.make_archive('/content/analysis', 'zip', 'experiment_checkpoints')
files.download('/content/analysis.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
# ====================================================================
#  DOWNLOAD RESULTS - WITH ERROR HANDLING
# ====================================================================

import shutil
import os
from google.colab import files

print(" Preparing downloads...")
print("="*70)

# Check what exists
results_exist = os.path.exists('./results')
checkpoints_exist = os.path.exists('./experiment_checkpoints')
epoch_checkpoints_exist = os.path.exists('./epoch_checkpoints')

print(f" Results directory: {' Found' if results_exist else ' Not found'}")
print(f" Experiment checkpoints: {' Found' if checkpoints_exist else ' Not found'}")
print(f" Epoch checkpoints: {' Found' if epoch_checkpoints_exist else ' Not found'}")
print()

downloads = []

# Create results archive if exists
if results_exist:
    print(" Creating results archive...")
    try:
        shutil.make_archive('/content/week_results', 'zip', './results')
        downloads.append('/content/week_results.zip')
        print("    Results archived")
    except Exception as e:
        print(f"     Results archive failed: {e}")
else:
    print("  No results directory found - no experiments completed yet")

# Create experiment checkpoints archive if exists
if checkpoints_exist and os.listdir('./experiment_checkpoints'):
    print(" Creating experiment checkpoints archive...")
    try:
        shutil.make_archive('/content/experiment_checkpoints', 'zip', './experiment_checkpoints')
        downloads.append('/content/experiment_checkpoints.zip')
        print("    Experiment checkpoints archived")
    except Exception as e:
        print(f"     Checkpoint archive failed: {e}")
else:
    print("  No experiment checkpoints found")

# Create epoch checkpoints archive if exists (optional - usually cleaned up)
if epoch_checkpoints_exist and os.listdir('./epoch_checkpoints'):
    print(" Creating epoch checkpoints archive...")
    try:
        shutil.make_archive('/content/epoch_checkpoints', 'zip', './epoch_checkpoints')
        downloads.append('/content/epoch_checkpoints.zip')
        print("    Epoch checkpoints archived")
    except Exception as e:
        print(f"     Epoch checkpoint archive failed: {e}")

# Download files
if downloads:
    print(f"\n  Starting downloads ({len(downloads)} files)...")
    print("="*70)
    for filepath in downloads:
        try:
            files.download(filepath)
            filename = os.path.basename(filepath)
            print(f"    Downloaded: {filename}")
        except Exception as e:
            print(f"    Download failed: {filepath} - {e}")
    print("\n Downloads complete!")
else:
    print("\n  Nothing to download - no completed experiments found")
    print("\n Tips:")
    print("   • Run training code first (Cell 6)")
    print("   • Wait for at least 1 experiment to complete")
    print("   • Then try downloading again")

print("="*70)

 Preparing downloads...
 Results directory:  Not found
 Experiment checkpoints:  Found
 Epoch checkpoints:  Found

  No results directory found - no experiments completed yet
 Creating experiment checkpoints archive...
    Experiment checkpoints archived
 Creating epoch checkpoints archive...
    Epoch checkpoints archived

  Starting downloads (2 files)...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

    Downloaded: experiment_checkpoints.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

    Downloaded: epoch_checkpoints.zip

 Downloads complete!
